In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist

# 加载 MNIST 数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 创建保存路径
import os
save_path = 'test_images'
os.makedirs(save_path, exist_ok=True)

# 保存一些图片为 PNG
def save_images(images, labels, indices):
    for i in indices:
        img = images[i]
        label = labels[i]
        file_name = f"{save_path}/{label}_{i}.png"
        plt.imsave(file_name, img, cmap='gray')
        print(f"保存图片: {file_name}")

# 从测试集里保存 3 张图片 (你可以指定其他索引)
save_images(x_test, y_test, [0, 3, 9])  # 保存索引 0, 1, 3 处的图片

保存图片: test_images/7_0.png
保存图片: test_images/0_3.png
保存图片: test_images/9_9.png


In [2]:
# 步骤1：数据的读取与预处理
import os
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data

class DataSource:
    def __init__(self):
        # 尝试直接加载数据集
        try:
            (x_train, y_train), (x_test, y_test) = load_data()
        except Exception as e:
            print("加载数据集时出错:", e)
            return

        # 增加一个通道维度
        x_train = x_train[..., tf.newaxis]
        x_test = x_test[..., tf.newaxis]

        # 像素值缩放到 [0, 1] 之间
        x_train, x_test = x_train / 255.0, x_test / 255.0

        self.train_images, self.train_labels = x_train, y_train
        self.test_images, self.test_labels = x_test, y_test

# 创建 DataSource 实例，加载数据
data = DataSource()

# 打印训练集和测试集的形状
print(f"训练集图片形状: {data.train_images.shape}")
print(f"训练集标签形状: {data.train_labels.shape}")
print(f"测试集图片形状: {data.test_images.shape}")
print(f"测试集标签形状: {data.test_labels.shape}")

# 验证是否为单通道
print(f"训练集单张图片通道数: {data.train_images.shape[-1]}")
print(f"测试集单张图片通道数: {data.test_images.shape[-1]}")


训练集图片形状: (60000, 28, 28, 1)
训练集标签形状: (60000,)
测试集图片形状: (10000, 28, 28, 1)
测试集标签形状: (10000,)
训练集单张图片通道数: 1
测试集单张图片通道数: 1


In [3]:
# 步骤2：搭建卷积神经网络（CNN模型）
from tensorflow.keras import layers, models

class CNN:
    def __init__(self):
        model = models.Sequential([
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.Flatten(),
            layers.Dense(64, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])

        # 打印模型结构
        model.summary()
        self.model = model


In [4]:
# 步骤3：训练模型、保存结果与可视化
import numpy as np
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
# 忽略警告
import warnings
warnings.filterwarnings('ignore', category=UserWarning)


class Train:
    def __init__(self):
        self.network = CNN()  # 创建CNN实例
        self.data = DataSource()  # 加载数据集

    def train(self):
        # 设置 TensorBoard 日志路径
        logdir = "./logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_cb = TensorBoard(log_dir=logdir)

        # # 设置模型检查点路径
        # check_path = './ckpt/cp-{epoch:04d}.ckpt'
        # save_model_cb = ModelCheckpoint(check_path, save_weights_only=True, verbose=1)
        # 设置模型检查点路径（每5个epoch保存一次）
        check_path = './ckpt/cp-{epoch:04d}.weights.h5'

        save_model_cb = tf.keras.callbacks.ModelCheckpoint(
            filepath=check_path,  # 修改文件后缀为 .weights.h5
            save_weights_only=True,
            verbose=1
        )


        # 编译模型
        self.network.model.compile(optimizer='adam',
                                   loss='sparse_categorical_crossentropy',
                                   metrics=['accuracy'])

        # 训练模型并保存日志
        training_history = self.network.model.fit(
            self.data.train_images,
            self.data.train_labels,
            epochs=10,
            validation_data=(self.data.test_images, self.data.test_labels),
            callbacks=[tensorboard_cb, save_model_cb]
        )

        # 保存模型为 .keras 格式
        os.makedirs('./keras', exist_ok=True)
        self.network.model.save('./keras/model.keras')
        print("模型已保存为 './keras/model.keras'")

        # 打印最终测试结果
        test_loss, test_acc = self.network.model.evaluate(self.data.test_images, 
                                                          self.data.test_labels)
        print(f"准确率：{test_acc * 100:.2f}%，共测试了 {len(self.data.test_labels)} 张图片")
        print("平均误差：", np.average(training_history.history['loss']))

if __name__ == "__main__":
    mnist_train = Train()
    mnist_train.train()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 3, 3, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,322 (364.54 KB)

 Trainable params: 93,322 (364.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8881 - loss: 0.3553
Epoch 1: saving model to ./ckpt/cp-0001.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.8882 - loss: 0.3552 - val_accuracy: 0.9825 - val_loss: 0.0539
Epoch 2/10
1874/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9847 - loss: 0.0492
Epoch 2: saving model to ./ckpt/cp-0002.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9847 - loss: 0.0492 - val_accuracy: 0.9904 - val_loss: 0.0311
Epoch 3/10
1870/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9907 - loss: 0.0315
Epoch 3: saving model to ./ckpt/cp-0003.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.9907 - loss: 0.0315 - val_accuracy: 0.9901 - val_loss: 0.0297
Epoch 4/10
1872/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9918 - loss: 0.0253
Epoch 4: saving model to ./ckpt/cp-0004.weights.h5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9918 - loss: 0.0253 - val_

In [5]:
# 步骤4：加载模型并进行预测
from PIL import Image

class Predict:
    def __init__(self):
        # 加载已训练的模型
        self.model = tf.keras.models.load_model('./keras/model.keras')

    def predict(self, image_path):
        # 读取并预处理图片
        img = Image.open(image_path).convert('L')
        img = img.resize((28, 28))  # 确保图片大小为 28x28
        img_array = np.array(img).reshape(1, 28, 28, 1) / 255.0  # 归一化

        # 进行预测
        prediction = self.model.predict(img_array)
        predicted_label = np.argmax(prediction[0])
        print(f"{image_path} -> 预测数字为：{predicted_label}")

# 测试预测
if __name__ == "__main__":
    app = Predict()
    app.predict('./test_images/7_0.png')
    app.predict('./test_images/0_3.png')
    app.predict('./test_images/9_9.png')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
./test_images/7_0.png -> 预测数字为：7
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
./test_images/0_3.png -> 预测数字为：0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
./test_images/9_9.png -> 预测数字为：9


In [6]:
!pip install matplotlib

In [7]:
!pip install google